In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
import time
import paho.mqtt.client as mqtt

/!\ Need to run publisher in other file

In [7]:

#  • The subscriber continuously receives the data.

# This function defines what to do when we connect to the broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("diabetesData")

last_five_records = []
num_correct = 0
num_message = 0
stop_loop = False

# This function defines what to do when we receives a message from the broker
def on_message(client, userdata, msg):
    global num_message, num_correct, last_five_records, stop_loop
    if num_message == 1000:
        print("Number of correct classifications after 1000 messages:", num_correct)
        stop_loop = True

    # Parse the received message to get the latest record
    latest_record = msg.payload.decode("utf-8")
    num_message += 1
    
    # Convert the latest record to a list of floats
    latest_record = list(map(float, latest_record.split(",")))
    
    # Add the latest record to the last_five_records list
    last_five_records.append(latest_record)
    
    # Keep only the last 5 records
    last_five_records = last_five_records[-5:]
    
    # Check if there are 5 records in last_five_records
    if len(last_five_records) == 5:
        # Perform the 3NN classification on the last 5 records
        predicted_outcome = perform_3nn_classification(last_five_records)
        
        # Compare the predicted outcome with the actual outcome in the latest record
        if predicted_outcome == latest_record[-1]:
            num_correct += 1

def perform_3nn_classification(last_five_records):
    # Extract the glucose, blood pressure, and insulin values from the last 5 records
    X = [[record[0], record[1], record[2]] for record in last_five_records]
    
    # Extract the outcome labels from the last 5 records
    y = [record[3] for record in last_five_records]
    
    # Normalize the glucose, blood pressure, and insulin values using MinMax
    scaler = MinMaxScaler()
    X_norm = scaler.fit_transform(X)
    
    # Perform the 3NN classification using the KNeighborsClassifier from scikit-learn
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_norm, y)
    predicted_outcome = knn.predict([X_norm[-1]])
    
    return predicted_outcome[0]

# We create a client as the data subscriber and specify its actions for particular events
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# Loop forever until stop_loop is True
while not stop_loop:
    mqttc.loop()

Connected with result code 0
Number of correct classifications after 1000 messages: 791
